<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-14 10:41:08
-------------------
qualified stocks: 94
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.03 L
Current:  1.28 C
-------------------
Today PnL: 79.85 K (0.63%)
Current PnL: -18.05 L (-12.8%)
CY Booked + Current PnL: -15.18 L (-10.76%)
-------------------
Total profit:  3.96 L
Total loss:  -22.01 L
-------------------
Total Booked + Current PnL: 11.80 L (10.35%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.25%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 79.93 L (62.63%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.42 C (110.94%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 8.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
53         MRF  114777.60  150504.14    14.94       X-MC  89.36  140945.0   
56  PIDILITIND    2504.06    3576.00   -15.79       X-LC  48.94   93327.0   
69     SBILIFE    1496.49    1924.99    -1.29       H-LC  80.85  146656.0   
70        SBIN     760.30     863.00   -12.51       M-LC  56.38  213352.0   
25       DMART    3484.00    5112.00   -21.08       X-LC  11.70   97152.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
53      26167.0   9556.0        0.37         22.80   6.78  31.13    215.0   
56      18205.0  13952.0        0.33         24.23  14.95  42.81     87.0   
69      20951.0  15047.0        0.34         16.67  10.26  28.63    116.0   
70      11872.0  15340.0        0.42          5.89   7.19  13.51     91.0   
25      13536.0  25532.0       -0.40         16.19  26.28  46.73    123.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
53     2.74        1.14    37.29       AR      ATH      TYRES  
56     1.30        0.76    17.04      X40      BTT  CHEMICALS  
69     1.39        1.19    30.65       AR      ATH  INSURANCE  
70     0.77        1.73    16.88     XY25      NTT      BANKS  
25     0.53        0.79    18.92       AR      NTT       FMCG

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
27   FINEORG  4771.46  7168.71   -20.50       H-SC  39.36   75752.0   
65      SAIL   130.64   228.00    81.22       M-MC  58.51  206519.0   
82  TITAGARH  1097.23  1548.00   -22.95       H-SC  15.96  135502.0   
6   ANGELONE  2328.67  3033.00     2.48       H-SC  76.60  166940.0   
75  SONACOMS   578.05  1006.00   -29.07       M-MC  85.11   91726.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
27      -5363.0   46118.0        3.67         -6.61  60.88  50.24    128.0   
65     -18443.0  186094.0        3.39         -8.20  90.11  74.53    217.0   
82     -56513.0  135394.0        3.36        -29.43  99.92  41.08     76.0   
6       15576.0   30199.0        3.24         10.29  18.09  30.25    131.0   
75      -9433.0   84324.0        3.15         -9.32  91.93  74.03     21.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy     Category  
27    -0.12        0.61    27.20     X40N      ATH    CHEMICALS  
65    -0.10        1.67    19.91     XY24      BTT        STEEL  
82    -0.42        1.10    14.69     XY24      NTT  ENGINEERING  
6      0.52        1.35    32.13     X40N      NTT      FINANCE  
75    -0.11        0.74    28.91       AR      BTT         AUTO

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
40        INFY  1432.81  2275.00   -12.42       X-LC  79.26  218035.0   
90      VIPIND   488.80   718.00  -786.44       H-SC  97.87   65422.0   
37   INDIAMART  2311.97  4911.36   -52.96       H-SC  68.09  101552.0   
7   ASIANPAINT  2961.56  4250.00   -21.82       X-LC   5.32  155502.0   
21      COLPAL  2837.00  3808.06     5.38       X-MC  40.43  128905.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
40      18874.0   98181.0       -3.58          9.48   45.03   58.78      9.0   
90     -29894.0   74588.0       -3.36        -31.36  114.01   46.89     51.0   
37       -175.0  114551.0       -1.80         -0.17  112.80  112.43      NaN   
7      -45884.0  133498.0       -1.63        -22.78   85.85   43.51    113.0   
21     -12945.0   61501.0       -0.80         -9.13   47.71   34.23     32.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
40     0.19        1.76    13.83      X40      BTT       IT  
90    -0.40        0.53    29.04     X40N      NTT     MISC  
37    -0.00        0.82    19.86       AR      ATH     MISC  
7     -0.34        1.26     7.26      X40      BTT   PAINTS  
21    -0.21        1.04     9.68      X40      ATH     FMCG

In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
80  TATAMOTORS   776.02  1065.00   -53.26       M-LC  84.04  186374.0   
2          ABB  5319.37  8953.12   -37.73       H-LC  74.47  142500.0   
24         DLF   683.35  1031.70   -24.28       H-LC  53.19  125388.0   
10         AWL   342.88   485.00   -68.61       H-MC   8.51  169831.0   
92   WHIRLPOOL  1151.42  2270.00   -35.54       M-SC  92.55  162484.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
80     -19271.0   95852.0       -0.62         -9.37  51.43  37.24      1.0   
2        9516.0   81325.0        1.13          7.16  57.07  68.31      2.0   
24       2385.0   60324.0        2.77          1.94  48.11  50.98      3.0   
10     -47898.0  138141.0        1.65        -22.00  81.34  41.45      5.0   
92      16254.0  125811.0        1.06         11.12  77.43  97.15      7.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
80    -0.20        1.51    21.31     XY24      NTT        AUTO  
2      0.12        1.15    16.20       AR      ATH  ELECTRICAL  
24     0.04        1.01    14.56       AR      ATH      REALTY  
10    -0.35        1.37    13.45     XY24      NTT        FMCG  
92     0.13        1.31    38.81      X40      NTT    DURABLES

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
53         MRF  114777.60  150504.14    14.94       X-MC  89.36  140945.0   
52     MEDANTA    1022.60    1486.00     7.25       X-MC  14.89  189027.0   
56  PIDILITIND    2504.06    3576.00   -15.79       X-LC  48.94   93327.0   
4   ADANIPORTS    1103.69    1583.00    -2.58       M-LC  87.23  188498.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
53      26167.0   9556.0        0.37         22.80   6.78  31.13    215.0   
52      33592.0  36841.0        0.16         21.61  19.49  45.32    198.0   
56      18205.0  13952.0        0.33         24.23  14.95  42.81     87.0   
4       37292.0  28369.0        0.49         24.66  15.05  43.43     37.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
53     2.74        1.14    37.29       AR      ATH       TYRES  
52     0.91        1.53    28.62     XY24      NTT  HEALTHCARE  
56     1.30        0.76    17.04      X40      BTT   CHEMICALS  
4      1.31        1.53    30.66     XY24      NTT        MISC

In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
50      LUXIND  1448.19  2442.00    33.16       M-SC  28.72  138310.0   
57     POLYCAB  5005.92  7541.35     6.97       H-MC  88.30  167216.0   
6     ANGELONE  2328.67  3033.00     2.48       H-SC  76.60  166940.0   
4   ADANIPORTS  1103.69  1583.00    -2.58       M-LC  87.23  188498.0   
93       WIPRO   243.46   420.00    -8.80       M-LC  45.74  156476.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
50        732.0   93677.0        2.02          0.53  67.73  68.62     55.0   
57      27050.0   43944.0       -0.59         19.30  26.28  50.65     54.0   
6       15576.0   30199.0        3.24         10.29  18.09  30.25    131.0   
4       37292.0   28369.0        0.49         24.66  15.05  43.43     37.0   
93       5531.0  103931.0        0.32          3.66  66.42  72.51     58.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
50     0.01        1.12    19.91     X40N      NTT  TEXTILES  
57     0.62        1.35    27.62     X40N      ATH    CABLES  
6      0.52        1.35    32.13     X40N      NTT   FINANCE  
4      1.31        1.53    30.66     XY24      NTT      MISC  
93     0.05        1.27    17.81       XR      NTT        IT

In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
93      WIPRO   243.46   420.00    -8.80       M-LC  45.74  156476.0   
12  BANKINDIA   113.49   190.00   -34.31       M-MC  37.23  129652.0   
37  INDIAMART  2311.97  4911.36   -52.96       H-SC  68.09  101552.0   
18       CAMS  3643.00  5250.99    -9.56       H-SC   6.38  101752.0   
91     VOLTAS  1272.00  1929.20   -30.74       H-MC  13.83  138413.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
93       5531.0  103931.0        0.32          3.66   66.42   72.51     58.0   
12        500.0   86569.0        1.22          0.39   66.77   67.42     71.0   
37       -175.0  114551.0       -1.80         -0.17  112.80  112.43      NaN   
18       -252.0   45280.0        0.22         -0.25   44.50   44.14     47.0   
91      -1507.0   73802.0        0.05         -1.08   53.32   51.67     19.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
93     0.05        1.27    17.81       XR      NTT       IT  
12     0.01        1.05    26.10       XR      NTT    BANKS  
37    -0.00        0.82    19.86       AR      ATH     MISC  
18    -0.01        0.82    15.95       XR      ATH     MISC  
91    -0.02        1.12     4.54       AR      ATH       AC

In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
85  UJJIVANSFB      52.77      60.00    37.69       M-SC  79.26  119583.0   
53         MRF  114777.60  150504.14    14.94       X-MC  89.36  140945.0   
69     SBILIFE    1496.49    1924.99    -1.29       H-LC  80.85  146656.0   
86   UNIONBANK     123.87     163.00   -12.58       M-LC  75.53  152904.0   
52     MEDANTA    1022.60    1486.00     7.25       X-MC  14.89  189027.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
85     -22896.0  42416.0        0.73        -16.07  35.47  13.70    219.0   
53      26167.0   9556.0        0.37         22.80   6.78  31.13    215.0   
69      20951.0  15047.0        0.34         16.67  10.26  28.63    116.0   
86      12064.0  32431.0        2.84          8.57  21.21  31.59     83.0   
52      33592.0  36841.0        0.16         21.61  19.49  45.32    198.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
85    -0.54        0.97    41.14     X40N      NTT       BANKS  
53     2.74        1.14    37.29       AR      ATH       TYRES  
69     1.39        1.19    30.65       AR      ATH   INSURANCE  
86     0.37        1.24    32.82     XY24      NTT       BANKS  
52     0.91        1.53    28.62     XY24      NTT  HEALTHCARE

In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
79  TATAELXSI  7332.28  9000.00   -18.67       X-MC  100.00   85358.0   
58      QUESS   711.66   986.00      NaN        NaN   98.94   80384.0   
90     VIPIND   488.80   718.00  -786.44       H-SC   97.87   65422.0   
1    AARTIIND   487.04   919.00    35.95       M-SC   96.81   78006.0   
78      TANLA   917.30  1963.11   -40.61       H-SC   95.74  145534.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
79     -17294.0   40639.0        1.85        -16.85   47.61   22.74     33.0   
58     -75470.0  135552.0        1.00        -48.42  168.63   38.55      NaN   
90     -29894.0   74588.0       -3.36        -31.36  114.01   46.89     51.0   
1       -3330.0   75471.0        0.09         -4.09   96.75   88.69    118.0   
78     -93881.0  366833.0        0.05        -39.21  252.06  114.01     29.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
79    -0.43        0.69    28.30     X40N      NTT         IT  
58    -0.56        0.65    23.09     XY24      NTT       MISC  
90    -0.40        0.53    29.04     X40N      NTT       MISC  
1     -0.04        0.63    31.54       XR      NTT  CHEMICALS  
78    -0.26        1.18    33.48       AR      ATH         IT

In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.27
1     25       40.97
2     50       69.53

In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.89
X40     14.40
X40N    14.05
AR      13.85
XR      10.30
XY25    10.22
X200     1.22
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.34
X-LC    18.22
M-SC    16.64
M-LC     8.44
H-MC     7.80
X-MC     6.76
H-LC     5.69
M-MC     5.54
X-SC     2.80
L-SC     2.26
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                11.74         -2.12   34.61
BANKS               11.14         -9.48   63.18
IT                   9.29        -20.70   88.04
FINANCE              7.49        -19.48   73.39
MISC                 7.17        -27.80   82.79
INSURANCE            6.10         -2.64   33.59
PAINTS               5.55        -20.14   55.69
ELECTRICAL           4.14         -8.72   64.12
REFINERIES           3.65          9.63    7.03
HEALTHCARE           3.64         -3.55   36.98
CHEMICALS            3.29        -16.02   95.96
AUTO                 3.19        -15.26   66.85
FOOTWEAR             2.43        -37.32   82.70
DURABLES             1.99         -0.42   78.42
TRAVEL               1.96        -31.80   88.84
AC                   1.92        -12.79   72.97
TEXTILES             1.69         -9.90   67.62
CERAMICS             1.67        -47.10  109.22
STEEL                1.67         -8.93   90.11
CABLES               1.35         16.18   26.28
JEWELLERY            1.27        -84.47  142.80
TYRES                1.14         18.57    6.78
CEMENT               1.10        -38.02  109.03
ENGINEERING          1.10        -41.71   99.92
PHARMA               1.08          1.94   46.26
REALTY               1.01          1.90   48.11
ENTERTAINMENT        0.91        -45.51  118.68
POWER                0.89        -28.91   80.32
HOTELS               0.46       -100.41  171.09

In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3041767.0     27
XR        1270870.0     15
AR        1245281.0     15
X40N      1195742.0     17
X40        813818.0     12
SR         209139.0      2
XY25       148569.0      5
X200        67491.0      1

In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2428984.0     22
M-SC        1949755.0     20
X-LC         732648.0     12
M-MC         582929.0      5
H-MC         431429.0      7
X-SC         362576.0      4
M-LC         338829.0      6
L-SC         318608.0      4
X-MC         318403.0      6
H-LC         222618.0      5

In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      926266.0      6
M-SC       XY24      767506.0      6
H-SC       X40N      555171.0      7
           AR        543149.0      3
M-SC       XR        431784.0      5
M-MC       XY24      412036.0      3
X-LC       X40       379834.0      5
M-SC       X40N      371347.0      5
H-SC       XR        303997.0      4
M-LC       XY24      219558.0      4
M-SC       X40       199328.0      2
H-LC       AR        188666.0      4
X-LC       AR        186046.0      3
H-MC       XY24      182146.0      2
L-SC       XR        164658.0      2
X-MC       XY24      139976.0      2
X-SC       SR        123518.0      1
           XR        121148.0      1
           X40N      117910.0      2
X-MC       X40N      107370.0      2
M-LC       XR        103931.0      1
H-SC       X40       100401.0      2
X-LC       XY25       99277.0      3
M-SC       AR         94169.0      1
L-SC       XY24       88381.0      1
M-MC       XR         86569.0      1
M-SC       SR         85621.0      1
M-MC       AR         84324.0      1
H-MC       AR         73802.0      1
           X40        72754.0      2
X-LC       X200       67491.0      1
L-SC       AR         65569.0      1
X-MC       X40        61501.0      1
H-MC       XR         58783.0      1
           X40N       43944.0      1
H-LC       XY25       33952.0      1
M-LC       XY25       15340.0      1
X-MC       AR          9556.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
